## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.metrics import accuracy_score
import xgboost as xgb

import pandas as pd
import tensorflow as tf
pd.set_option('display.max_rows', None)

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(columns=['EIN','NAME', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT'])
application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,1


In [3]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df.nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
STATUS               2
INCOME_AMT           9
IS_SUCCESSFUL        2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T13','T12','T2','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
class_values = application_df['CLASSIFICATION'].value_counts()

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_values[class_values < 1883].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = [
    'APPLICATION_TYPE', 
    'AFFILIATION', 
    'CLASSIFICATION', 
    'USE_CASE', 
    'ORGANIZATION', 
    'INCOME_AMT', 
]

application_df_encoded = pd.get_dummies(application_df, columns=categorical_columns,drop_first=True)
application_df_encoded.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,1,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,1,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,1,0,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1,1,False,False,True,False,False,False,False,False,...,False,True,False,True,False,False,False,False,False,False
4,1,1,False,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,False,False,False


In [9]:
# Split our preprocessed data into our features and target arrays
y = application_df_encoded['IS_SUCCESSFUL']

X = application_df_encoded.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)



In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

Optimization Attempt #1

In [11]:
# Define the model - deep neural net
nn = tf.keras.models.Sequential()

# Input Layer 
nn.add(tf.keras.layers.Dense(units=64, activation='relu', input_dim=X_train_scaled.shape[1]))

# First hidden layer 
nn.add(tf.keras.layers.Dense(units=32, activation='relu'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Output layer with sigmoid activation (binary classification)
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

c:\Users\Shari\Desktop\Data Science Course\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,929 (19.25 KB)

 Trainable params: 4,929 (19.25 KB)

 Non-trainable params: 0 (0.00 B)

Optimization Attempt #2: Add L2 Regulizers and Dropout Layers
L1/L2 Regularization: This can be added to the weights of the layers to prevent overfitting by penalizing large weights.
Dropout layers are a regularization technique used in neural networks to prevent overfitting. During training, dropout temporarily "drops out" (sets to zero) a random subset of neurons in a layer at each training iteration.


In [12]:
# Define the model
nn2 = tf.keras.models.Sequential()
# Input Layer
nn2.add(tf.keras.layers.Dense(units=64, activation='relu', input_dim=X_train_scaled.shape[1],
                             kernel_regularizer=tf.keras.regularizers.l2(0.001)))
nn2.add(tf.keras.layers.Dropout(0.2))

# Second Layer
nn2.add(tf.keras.layers.Dense(units=32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
nn2.add(tf.keras.layers.Dropout(0.2))

# Output Layer
nn2.add(tf.keras.layers.Dense(units=16, activation='relu'))
nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 64)             │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,929 (19.25 KB)

 Trainable params: 4,929 (19.25 KB)

 Non-trainable params: 0 (0.00 B)

Optimization Attempt #3: Added more perceptrons in each layer and used a different optimzer

In [13]:
# Define the model - deep neural net
nn3 = tf.keras.models.Sequential()

# Input Layer 
nn3.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))

# First hidden layer 
nn3.add(tf.keras.layers.Dense(units=64, activation='relu'))
# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=32, activation='relu'))
# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Output layer with sigmoid activation (binary classification)
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 128)            │         4,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,489 (60.50 KB)

 Trainable params: 15,489 (60.50 KB)

 Non-trainable params: 0 (0.00 B)

Each optimizer correponds with the each attempt

In [14]:
# Compile Model 1
optimizer1 = Adam(learning_rate=0.001)
nn.compile(optimizer=optimizer1,
           loss='binary_crossentropy',
           metrics=['accuracy'])

In [15]:
# Compile Model 2 with adjusted learning rate
optimizer2 = Adam(learning_rate=0.005)
nn2.compile(optimizer=optimizer2,
           loss='binary_crossentropy',
           metrics=['accuracy'])

In [16]:
# Compile Model 3 with different optimzer and learning rate
optimizer3 = RMSprop(learning_rate=0.003)
nn3.compile(optimizer=optimizer3,
           loss='binary_crossentropy',
           metrics=['accuracy'])

In [17]:

# Train the model
fit_model = nn.fit(
    X_train_scaled, y_train,
    epochs=33, # Number of iterations
    batch_size=32, # Number of samples per gradient update
)

Epoch 1/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 891us/step - accuracy: 0.7027 - loss: 0.5955
Epoch 2/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 930us/step - accuracy: 0.7216 - loss: 0.5596
Epoch 3/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 863us/step - accuracy: 0.7272 - loss: 0.5560
Epoch 4/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 818us/step - accuracy: 0.7339 - loss: 0.5511
Epoch 5/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 802us/step - accuracy: 0.7346 - loss: 0.5461
Epoch 6/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 852us/step - accuracy: 0.7310 - loss: 0.5484
Epoch 7/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 870us/step - accuracy: 0.7336 - loss: 0.5456
Epoch 8/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 921us/step - accuracy: 0.7301 - loss: 0.5452
Epoch 9/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 860us/step - accuracy: 0.7338 - loss: 0.5446
Epoch 10/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 878us/step - accuracy: 0.7334 - loss: 0.5478
Epoch 11/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 918us/step - accuracy: 0.7308 - loss: 0.5469
Epoch 12/33
804/804 ━━━━━━━━━━

In [18]:
# Train the model
fit_model2 = nn2.fit(
    X_train_scaled, y_train,
    epochs=33, # Number of iterations
    batch_size=32, # Number of samples per gradient update
)

Epoch 1/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6898 - loss: 0.6518
Epoch 2/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7225 - loss: 0.5835
Epoch 3/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7191 - loss: 0.5865
Epoch 4/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7254 - loss: 0.5815
Epoch 5/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7170 - loss: 0.5845
Epoch 6/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 991us/step - accuracy: 0.7228 - loss: 0.5802
Epoch 7/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 995us/step - accuracy: 0.7295 - loss: 0.5751
Epoch 8/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 998us/step - accuracy: 0.7242 - loss: 0.5787
Epoch 9/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 986us/step - accuracy: 0.7238 - loss: 0.5775
Epoch 10/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 970us/step - accuracy: 0.7265 - loss: 0.5762
Epoch 11/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 976us/step - accuracy: 0.7255 - loss: 0.5755
Epoch 12/33
804/804 ━━━━━━━━━━━━━━━━━━━━

In [19]:
# Train the model
fit_model3 = nn3.fit(
    X_train_scaled, y_train,
    epochs=33, # Number of iterations
    batch_size=32, # Number of samples per gradient update
)

Epoch 1/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7075 - loss: 0.5885
Epoch 2/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 985us/step - accuracy: 0.7264 - loss: 0.5643
Epoch 3/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 958us/step - accuracy: 0.7265 - loss: 0.5636
Epoch 4/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 976us/step - accuracy: 0.7293 - loss: 0.5609
Epoch 5/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 974us/step - accuracy: 0.7294 - loss: 0.5579
Epoch 6/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 949us/step - accuracy: 0.7356 - loss: 0.5516
Epoch 7/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 971us/step - accuracy: 0.7325 - loss: 0.5531
Epoch 8/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 985us/step - accuracy: 0.7328 - loss: 0.5550
Epoch 9/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 944us/step - accuracy: 0.7269 - loss: 0.5605
Epoch 10/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 993us/step - accuracy: 0.7340 - loss: 0.5582
Epoch 11/33
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7310 - loss: 0.5575
Epoch 12/33
804/804 ━━━━━━━━━━━━━━

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7290 - loss: 0.5514
Loss: 0.5513527393341064, Accuracy: 0.7289795875549316


In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7277 - loss: 0.5755
Loss: 0.5754640698432922, Accuracy: 0.7276967763900757


In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7243 - loss: 0.5997
Loss: 0.599687397480011, Accuracy: 0.7243148684501648


In [24]:
# Export our model to HDF5 file (Attempt 1)
nn.save('AlphabetSoupCharity_Optimization.h5')
# OPTIONAL:Save model using native Keras format
nn.save('AlphabetSoupCharity_Optimization.keras')

# Export our model to HDF5 file (Attempt 2)
nn2.save('AlphabetSoupCharity_Optimization2.h5')
# OPTIONAL:Save model using native Keras format
nn2.save('AlphabetSoupCharity_Optimization2.keras')# Export our model to HDF5 file

# Export our model to HDF5 file (Attempt 3)
nn3.save('AlphabetSoupCharity_Optimization3.h5')
# OPTIONAL:Save model using native Keras format
nn3.save('AlphabetSoupCharity_Optimization3.keras')